# 拡散モデル訓練可視化ノートブック

このノートブックでは、個人最適化軌道生成のための拡散モデルの訓練プロセスと結果を可視化します。

## 内容
1. 環境設定
2. モデルアーキテクチャの可視化
3. ノイズスケジュールの分析
4. 訓練プロセスの可視化
5. 生成プロセスの段階的可視化
6. 個人特性と軌道の関係分析
7. 結果の評価

In [ ]:
# CLAUDE_ADDED
import sys
import os

# プロジェクトルートをパスに追加
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import pandas as pd
from typing import Dict, List, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# ローカルモジュールのインポート
from Model import UNet1D
from train import DDPMScheduler, TrajectoryDataset, DiffusionTrainer, create_dummy_data
from generate import TrajectoryGenerator, load_model

# プロット設定
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 1. 環境設定と基本パラメータ

In [ ]:
# CLAUDE_ADDED
# デバイス設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用デバイス: {device}')

# 基本パラメータ
CONFIG = {
    'input_dim': 2,          # 軌道次元 (x, y)
    'condition_dim': 5,      # 個人特性次元 (動作時間、終点誤差、ジャーク等)
    'seq_len': 101,          # 軌道の長さ
    'time_embed_dim': 128,   # 時間埋め込み次元
    'base_channels': 64,     # ベースチャンネル数
    'num_timesteps': 1000,   # 拡散ステップ数
    'batch_size': 32,        # バッチサイズ
    'learning_rate': 1e-4,   # 学習率
}

print('設定パラメータ:')
for key, value in CONFIG.items():
    print(f'  {key}: {value}')

## 2. モデルアーキテクチャの可視化

In [ ]:
# CLAUDE_ADDED
# モデルの作成
model = UNet1D(
    input_dim=CONFIG['input_dim'],
    condition_dim=CONFIG['condition_dim'],
    time_embed_dim=CONFIG['time_embed_dim'],
    base_channels=CONFIG['base_channels']
).to(device)

# パラメータ数の計算
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'総パラメータ数: {total_params:,}')
print(f'学習可能パラメータ数: {trainable_params:,}')

# レイヤー別パラメータ数の分析
layer_info = []
for name, module in model.named_modules():
    if len(list(module.children())) == 0:  # 末端のモジュールのみ
        params = sum(p.numel() for p in module.parameters())
        if params > 0:
            layer_info.append({
                'Layer': name,
                'Parameters': params,
                'Percentage': (params / total_params) * 100
            })

# パラメータ数の可視化
layer_df = pd.DataFrame(layer_info).sort_values('Parameters', ascending=False).head(10)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# バープロット
ax1.barh(layer_df['Layer'][::-1], layer_df['Parameters'][::-1])
ax1.set_xlabel('パラメータ数')
ax1.set_title('レイヤー別パラメータ数（上位10層）')
ax1.grid(True, alpha=0.3)

# 円グラフ
others_params = total_params - layer_df['Parameters'].sum()
pie_data = list(layer_df['Parameters'].head(5)) + [others_params]
pie_labels = list(layer_df['Layer'].head(5)) + ['Others']

ax2.pie(pie_data, labels=pie_labels, autopct='%1.1f%%', startangle=90)
ax2.set_title('パラメータ分布（上位5層 + その他）')

plt.tight_layout()
plt.show()

## 3. ノイズスケジュールの分析

In [ ]:
# CLAUDE_ADDED
# DDPMスケジューラの作成
scheduler = DDPMScheduler(num_timesteps=CONFIG['num_timesteps'])

# タイムステップ
timesteps = np.arange(CONFIG['num_timesteps'])

# スケジュールパラメータの取得
betas = scheduler.betas.numpy()
alphas = scheduler.alphas.numpy()
alphas_cumprod = scheduler.alphas_cumprod.numpy()
sqrt_alphas_cumprod = scheduler.sqrt_alphas_cumprod.numpy()
sqrt_one_minus_alphas_cumprod = scheduler.sqrt_one_minus_alphas_cumprod.numpy()

# 可視化
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# β スケジュール
axes[0, 0].plot(timesteps, betas, linewidth=2)
axes[0, 0].set_title('β スケジュール')
axes[0, 0].set_xlabel('タイムステップ')
axes[0, 0].set_ylabel('β 値')
axes[0, 0].grid(True, alpha=0.3)

# α スケジュール
axes[0, 1].plot(timesteps, alphas, linewidth=2)
axes[0, 1].set_title('α スケジュール')
axes[0, 1].set_xlabel('タイムステップ')
axes[0, 1].set_ylabel('α 値')
axes[0, 1].grid(True, alpha=0.3)

# α 累積積
axes[0, 2].plot(timesteps, alphas_cumprod, linewidth=2)
axes[0, 2].set_title('α 累積積')
axes[0, 2].set_xlabel('タイムステップ')
axes[0, 2].set_ylabel('α̅ 値')
axes[0, 2].grid(True, alpha=0.3)

# 信号強度
axes[1, 0].plot(timesteps, sqrt_alphas_cumprod, label='信号強度', linewidth=2)
axes[1, 0].plot(timesteps, sqrt_one_minus_alphas_cumprod, label='ノイズ強度', linewidth=2)
axes[1, 0].set_title('信号とノイズの強度')
axes[1, 0].set_xlabel('タイムステップ')
axes[1, 0].set_ylabel('強度')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# SNR (Signal-to-Noise Ratio)
snr = alphas_cumprod / (1 - alphas_cumprod)
axes[1, 1].semilogy(timesteps, snr, linewidth=2)
axes[1, 1].set_title('信号対雑音比 (SNR)')
axes[1, 1].set_xlabel('タイムステップ')
axes[1, 1].set_ylabel('SNR (log scale)')
axes[1, 1].grid(True, alpha=0.3)

# ノイズレベルの分布
noise_levels = sqrt_one_minus_alphas_cumprod
axes[1, 2].hist(noise_levels, bins=50, alpha=0.7, edgecolor='black')
axes[1, 2].set_title('ノイズレベルの分布')
axes[1, 2].set_xlabel('ノイズレベル')
axes[1, 2].set_ylabel('頻度')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 統計情報
print(f'β 範囲: [{betas.min():.6f}, {betas.max():.6f}]')
print(f'最終ノイズレベル: {sqrt_one_minus_alphas_cumprod[-1]:.6f}')
print(f'最終SNR: {snr[-1]:.6f}')

## 4. 訓練データの準備と可視化

In [ ]:
# CLAUDE_ADDED
# ダミーデータの生成
print('訓練データを生成中...')
train_trajectories, train_conditions = create_dummy_data(
    num_samples=1000, 
    seq_len=CONFIG['seq_len'], 
    condition_dim=CONFIG['condition_dim']
)

val_trajectories, val_conditions = create_dummy_data(
    num_samples=200, 
    seq_len=CONFIG['seq_len'], 
    condition_dim=CONFIG['condition_dim']
)

# データセットとデータローダーの作成
train_dataset = TrajectoryDataset(train_trajectories, train_conditions)
val_dataset = TrajectoryDataset(val_trajectories, val_conditions)

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False)

print(f'訓練データ: {len(train_dataset)} サンプル')
print(f'バリデーションデータ: {len(val_dataset)} サンプル')

# サンプルデータの可視化
sample_batch = next(iter(train_loader))
sample_trajectories, sample_conditions = sample_batch

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i in range(8):
    row = i // 4
    col = i % 4
    ax = axes[row, col]
    
    traj = sample_trajectories[i].numpy()
    condition = sample_conditions[i].numpy()
    
    ax.plot(traj[0], traj[1], 'b-', linewidth=2, alpha=0.8)
    ax.plot(traj[0, 0], traj[1, 0], 'go', markersize=8, label='Start')
    ax.plot(traj[0, -1], traj[1, -1], 'ro', markersize=8, label='End')
    
    ax.set_title(f'軌道 {i+1}\n条件: [{condition[0]:.2f}, {condition[1]:.2f}, {condition[2]:.2f}...]')
    ax.set_xlabel('X 位置')
    ax.set_ylabel('Y 位置')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()

## 5. ノイズ追加プロセスの可視化

In [ ]:
# CLAUDE_ADDED
# サンプル軌道を選択
sample_traj = sample_trajectories[0:1]  # 最初の軌道を選択
sample_cond = sample_conditions[0:1]

# 異なるタイムステップでのノイズ追加を可視化
timesteps_to_show = [0, 100, 300, 500, 700, 900, 999]
num_steps = len(timesteps_to_show)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

original_traj = sample_traj[0].numpy()

for i, t in enumerate(timesteps_to_show):
    if i >= len(axes):
        break
        
    ax = axes[i]
    
    # ノイズを追加
    timestep_tensor = torch.tensor([t])
    noise = torch.randn_like(sample_traj)
    noisy_traj = scheduler.add_noise(sample_traj, noise, timestep_tensor)
    noisy_traj_np = noisy_traj[0].numpy()
    
    # 元の軌道
    ax.plot(original_traj[0], original_traj[1], 'b-', linewidth=3, alpha=0.7, label='Original')
    
    # ノイズ追加後の軌道
    ax.plot(noisy_traj_np[0], noisy_traj_np[1], 'r--', linewidth=2, alpha=0.8, label=f'Noisy (t={t})')
    
    # 開始点と終了点
    ax.plot(original_traj[0, 0], original_traj[1, 0], 'go', markersize=8)
    ax.plot(original_traj[0, -1], original_traj[1, -1], 'bo', markersize=8)
    
    # ノイズレベルの表示
    noise_level = sqrt_one_minus_alphas_cumprod[t]
    signal_level = sqrt_alphas_cumprod[t]
    
    ax.set_title(f't = {t}\nノイズ: {noise_level:.3f}, 信号: {signal_level:.3f}')
    ax.set_xlabel('X 位置')
    ax.set_ylabel('Y 位置')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_aspect('equal')

# 空のサブプロットを非表示
for i in range(num_steps, len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

# ノイズレベルと信号レベルの関係をプロット
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.plot(timesteps_to_show, [sqrt_alphas_cumprod[t] for t in timesteps_to_show], 
        'b-o', linewidth=2, markersize=8, label='信号レベル')
ax.plot(timesteps_to_show, [sqrt_one_minus_alphas_cumprod[t] for t in timesteps_to_show], 
        'r-o', linewidth=2, markersize=8, label='ノイズレベル')

ax.set_xlabel('タイムステップ')
ax.set_ylabel('レベル')
ax.set_title('ノイズ追加プロセスにおける信号とノイズのレベル')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

## 6. モデル訓練の実行と可視化

In [ ]:
# CLAUDE_ADDED
# 訓練器の作成
trainer = DiffusionTrainer(
    model=model, 
    scheduler=scheduler, 
    device=device, 
    learning_rate=CONFIG['learning_rate']
)

# 短期間の訓練実行（デモ用）
print('デモ用短期訓練を開始...')
train_losses = []
val_losses = []

num_demo_epochs = 5  # デモ用に短縮

for epoch in range(num_demo_epochs):
    model.train()
    epoch_loss = 0.0
    
    # 訓練ループ
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_demo_epochs}', leave=False)
    for batch in progress_bar:
        loss = trainer.train_step(batch)
        epoch_loss += loss
        progress_bar.set_postfix({'Loss': f'{loss:.4f}'})
    
    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # バリデーション
    val_loss = trainer.validate(val_loader)
    val_losses.append(val_loss)
    
    print(f'Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {val_loss:.4f}')

print('デモ訓練完了!')

In [ ]:
# CLAUDE_ADDED
# 訓練曲線の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

epochs = range(1, len(train_losses) + 1)

# 損失曲線
ax1.plot(epochs, train_losses, 'b-o', linewidth=2, markersize=6, label='訓練損失')
ax1.plot(epochs, val_losses, 'r-o', linewidth=2, markersize=6, label='バリデーション損失')
ax1.set_xlabel('エポック')
ax1.set_ylabel('損失')
ax1.set_title('訓練曲線')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 損失の改善率
if len(train_losses) > 1:
    train_improvement = [(train_losses[i-1] - train_losses[i]) / train_losses[i-1] * 100 
                        for i in range(1, len(train_losses))]
    val_improvement = [(val_losses[i-1] - val_losses[i]) / val_losses[i-1] * 100 
                      for i in range(1, len(val_losses))]
    
    ax2.plot(epochs[1:], train_improvement, 'b-o', linewidth=2, markersize=6, label='訓練改善率')
    ax2.plot(epochs[1:], val_improvement, 'r-o', linewidth=2, markersize=6, label='バリデーション改善率')
    ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5)
    ax2.set_xlabel('エポック')
    ax2.set_ylabel('改善率 (%)')
    ax2.set_title('損失改善率')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 最終統計
print(f'\n訓練統計:')
print(f'  最終訓練損失: {train_losses[-1]:.4f}')
print(f'  最終バリデーション損失: {val_losses[-1]:.4f}')
if len(train_losses) > 1:
    total_train_improvement = (train_losses[0] - train_losses[-1]) / train_losses[0] * 100
    total_val_improvement = (val_losses[0] - val_losses[-1]) / val_losses[0] * 100
    print(f'  訓練損失改善: {total_train_improvement:.2f}%')
    print(f'  バリデーション損失改善: {total_val_improvement:.2f}%')

## 7. 軌道生成と逆拡散プロセスの可視化

In [ ]:
# CLAUDE_ADDED
# 軌道ジェネレータの作成
generator = TrajectoryGenerator(model, scheduler, device)

# サンプル条件を準備
sample_conditions = torch.randn(1, CONFIG['condition_dim']).to(device)

# 逆拡散プロセスの段階的可視化
print('逆拡散プロセスを可視化中...')

# 特定のタイムステップでの中間結果を保存
timesteps_to_visualize = [999, 800, 600, 400, 200, 100, 50, 20, 0]
intermediate_results = []

with torch.no_grad():
    shape = (1, 2, CONFIG['seq_len'])
    x = torch.randn(shape, device=device)
    
    for t in range(scheduler.num_timesteps-1, -1, -1):
        if t in timesteps_to_visualize:
            intermediate_results.append((t, x.clone()))
        
        t_tensor = torch.full((1,), t, device=device, dtype=torch.long)
        predicted_noise = model(x, t_tensor, sample_conditions)
        
        if t > 0:
            sqrt_recip_alpha_t = generator.sqrt_recip_alphas[t]
            beta_t = scheduler.betas[t]
            sqrt_one_minus_alphas_cumprod_t = generator.sqrt_one_minus_alphas_cumprod[t]
            
            mean = sqrt_recip_alpha_t * (x - beta_t / sqrt_one_minus_alphas_cumprod_t * predicted_noise)
            
            alpha_t = scheduler.alphas[t]
            alpha_cumprod_t = scheduler.alphas_cumprod[t]
            alpha_cumprod_prev = scheduler.alphas_cumprod_prev[t]
            
            variance = beta_t * (1 - alpha_cumprod_prev) / (1 - alpha_cumprod_t)
            sigma = torch.sqrt(variance)
            noise = torch.randn_like(x)
            
            x = mean + sigma * noise
        else:
            sqrt_recip_alpha_t = generator.sqrt_recip_alphas[t]
            beta_t = scheduler.betas[t]
            sqrt_one_minus_alphas_cumprod_t = generator.sqrt_one_minus_alphas_cumprod[t]
            x = sqrt_recip_alpha_t * (x - beta_t / sqrt_one_minus_alphas_cumprod_t * predicted_noise)

    # 最終結果も追加
    if 0 not in [result[0] for result in intermediate_results]:
        intermediate_results.append((0, x.clone()))

In [ ]:
# CLAUDE_ADDED
# 逆拡散プロセスの可視化
num_steps = len(intermediate_results)
cols = 3
rows = (num_steps + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
if rows == 1:
    axes = axes.reshape(1, -1)

for idx, (timestep, trajectory) in enumerate(intermediate_results):
    row = idx // cols
    col = idx % cols
    ax = axes[row, col]
    
    traj_np = trajectory[0].cpu().numpy()
    
    # 軌道をプロット
    ax.plot(traj_np[0], traj_np[1], 'b-', linewidth=2, alpha=0.8)
    ax.plot(traj_np[0, 0], traj_np[1, 0], 'go', markersize=10, label='Start')
    ax.plot(traj_np[0, -1], traj_np[1, -1], 'ro', markersize=10, label='End')
    
    # タイトルと統計情報
    noise_level = sqrt_one_minus_alphas_cumprod[timestep] if timestep < len(sqrt_one_minus_alphas_cumprod) else 0
    mean_x, std_x = traj_np[0].mean(), traj_np[0].std()
    mean_y, std_y = traj_np[1].mean(), traj_np[1].std()
    
    ax.set_title(f't = {timestep}\nノイズレベル: {noise_level:.3f}\nX: μ={mean_x:.2f}, σ={std_x:.2f}\nY: μ={mean_y:.2f}, σ={std_y:.2f}')
    ax.set_xlabel('X 位置')
    ax.set_ylabel('Y 位置')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_aspect('equal')

# 空のサブプロットを非表示
for idx in range(num_steps, rows * cols):
    row = idx // cols
    col = idx % cols
    axes[row, col].set_visible(False)

plt.tight_layout()
plt.show()

## 8. 複数軌道の生成と個人特性の影響分析

In [ ]:
# CLAUDE_ADDED
# 異なる個人特性での軌道生成
print('異なる個人特性での軌道生成中...')

# 多様な個人特性を生成
num_conditions = 8
varied_conditions = torch.randn(num_conditions, CONFIG['condition_dim']).to(device)

# 特性を意図的に変化させる（デモ用）
for i in range(num_conditions):
    # 第1特性：動作時間（-2から2まで変化）
    varied_conditions[i, 0] = -2 + 4 * i / (num_conditions - 1)
    # 第2特性：終点誤差（-1.5から1.5まで変化）
    varied_conditions[i, 1] = -1.5 + 3 * i / (num_conditions - 1)

# 軌道生成
with torch.no_grad():
    generated_trajectories = generator.generate_trajectories(
        conditions=varied_conditions,
        seq_len=CONFIG['seq_len'],
        method='ddim',
        num_inference_steps=50
    )

# 生成された軌道の可視化
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i in range(num_conditions):
    ax = axes[i]
    
    traj = generated_trajectories[i].cpu().numpy()
    condition = varied_conditions[i].cpu().numpy()
    
    ax.plot(traj[0], traj[1], 'b-', linewidth=2, alpha=0.8)
    ax.plot(traj[0, 0], traj[1, 0], 'go', markersize=10, label='Start')
    ax.plot(traj[0, -1], traj[1, -1], 'ro', markersize=10, label='End')
    
    # 軌道統計
    total_distance = np.sum(np.sqrt(np.sum(np.diff(traj, axis=1)**2, axis=0)))
    end_error = np.sqrt(np.sum(traj[:, -1]**2))
    
    ax.set_title(f'条件 {i+1}\n特性1: {condition[0]:.2f}, 特性2: {condition[1]:.2f}\n距離: {total_distance:.2f}, 終点誤差: {end_error:.2f}')
    ax.set_xlabel('X 位置')
    ax.set_ylabel('Y 位置')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()

In [ ]:
# CLAUDE_ADDED
# 軌道特徴量の抽出と分析
def extract_trajectory_features(trajectories_tensor):
    """
    軌道から特徴量を抽出
    """
    trajectories = trajectories_tensor.cpu().numpy()
    features = {}
    
    # 移動距離
    distances = np.sqrt(np.sum(np.diff(trajectories, axis=2)**2, axis=1))
    features['total_distance'] = np.sum(distances, axis=1)
    
    # 最大速度
    velocities = np.sqrt(np.sum(np.diff(trajectories, axis=2)**2, axis=1))
    features['max_velocity'] = np.max(velocities, axis=1)
    
    # 平均速度
    features['mean_velocity'] = np.mean(velocities, axis=1)
    
    # 終点誤差（原点からの距離）
    end_points = trajectories[:, :, -1]
    features['end_point_error'] = np.sqrt(np.sum(end_points**2, axis=1))
    
    # 軌道の滑らかさ（加速度の変化）
    accelerations = np.diff(velocities, axis=1)
    jerks = np.diff(accelerations, axis=1)
    features['jerk_mean'] = np.mean(np.abs(jerks), axis=1)
    features['jerk_max'] = np.max(np.abs(jerks), axis=1)
    
    # 軌道の範囲
    features['x_range'] = np.ptp(trajectories[:, 0, :], axis=1)
    features['y_range'] = np.ptp(trajectories[:, 1, :], axis=1)
    
    # 軌道の曲率（方向変化）
    dx = np.diff(trajectories[:, 0, :], axis=1)
    dy = np.diff(trajectories[:, 1, :], axis=1)
    angles = np.arctan2(dy, dx)
    angle_changes = np.abs(np.diff(angles, axis=1))
    features['curvature_mean'] = np.mean(angle_changes, axis=1)
    features['curvature_max'] = np.max(angle_changes, axis=1)
    
    return features

# 特徴量の抽出
trajectory_features = extract_trajectory_features(generated_trajectories)
conditions_np = varied_conditions.cpu().numpy()

# 相関分析
feature_names = list(trajectory_features.keys())
condition_names = [f'条件_{i+1}' for i in range(CONFIG['condition_dim'])]

# データフレーム作成
feature_df = pd.DataFrame(trajectory_features)
condition_df = pd.DataFrame(conditions_np, columns=condition_names)
combined_df = pd.concat([condition_df, feature_df], axis=1)

# 相関行列
correlation_matrix = combined_df.corr()
condition_feature_corr = correlation_matrix.loc[condition_names, feature_names]

# 相関ヒートマップ
fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(condition_feature_corr, 
           annot=True, 
           cmap='RdBu_r', 
           center=0, 
           square=True,
           linewidths=0.5,
           ax=ax)
ax.set_title('個人特性と軌道特徴の相関関係')
ax.set_xlabel('軌道特徴')
ax.set_ylabel('個人特性')
plt.tight_layout()
plt.show()

# 統計サマリー
print('\n軌道特徴の統計サマリー:')
print(feature_df.describe())

## 9. サンプリング手法の比較

In [ ]:
# CLAUDE_ADDED
# DDPM vs DDIM の比較
print('異なるサンプリング手法の比較...')

sample_condition = varied_conditions[:1]  # 同じ条件を使用

# DDPM サンプリング（100ステップ）
with torch.no_grad():
    ddpm_trajectory = generator.generate_trajectories(
        conditions=sample_condition,
        method='ddpm',
        num_inference_steps=100
    )

# DDIM サンプリング（異なるステップ数）
ddim_steps = [10, 25, 50, 100]
ddim_trajectories = []

for steps in ddim_steps:
    with torch.no_grad():
        ddim_traj = generator.generate_trajectories(
            conditions=sample_condition,
            method='ddim',
            num_inference_steps=steps
        )
        ddim_trajectories.append(ddim_traj)

# 可視化
fig, axes = plt.subplots(1, 5, figsize=(25, 5))

# DDPM結果
ddpm_traj_np = ddpm_trajectory[0].cpu().numpy()
axes[0].plot(ddpm_traj_np[0], ddmp_traj_np[1], 'b-', linewidth=2, alpha=0.8)
axes[0].plot(ddpm_traj_np[0, 0], ddpm_traj_np[1, 0], 'go', markersize=10)
axes[0].plot(ddpm_traj_np[0, -1], ddpm_traj_np[1, -1], 'ro', markersize=10)
axes[0].set_title('DDPM\n(100 steps)')
axes[0].set_xlabel('X 位置')
axes[0].set_ylabel('Y 位置')
axes[0].grid(True, alpha=0.3)
axes[0].set_aspect('equal')

# DDIM結果
for i, (steps, ddim_traj) in enumerate(zip(ddim_steps, ddim_trajectories)):
    ddim_traj_np = ddim_traj[0].cpu().numpy()
    axes[i+1].plot(ddim_traj_np[0], ddim_traj_np[1], 'r-', linewidth=2, alpha=0.8)
    axes[i+1].plot(ddim_traj_np[0, 0], ddim_traj_np[1, 0], 'go', markersize=10)
    axes[i+1].plot(ddim_traj_np[0, -1], ddim_traj_np[1, -1], 'ro', markersize=10)
    axes[i+1].set_title(f'DDIM\n({steps} steps)')
    axes[i+1].set_xlabel('X 位置')
    axes[i+1].set_ylabel('Y 位置')
    axes[i+1].grid(True, alpha=0.3)
    axes[i+1].set_aspect('equal')

plt.tight_layout()
plt.show()

# 軌道品質の定量比較
methods_trajectories = [ddpm_trajectory] + ddim_trajectories
method_names = ['DDPM (100)'] + [f'DDIM ({steps})' for steps in ddim_steps]

quality_metrics = []
for traj in methods_trajectories:
    features = extract_trajectory_features(traj)
    quality_metrics.append({
        'total_distance': features['total_distance'][0],
        'smoothness': 1.0 / (features['jerk_mean'][0] + 1e-6),  # 逆数で滑らかさを表現
        'end_error': features['end_point_error'][0]
    })

# 品質メトリクスの可視化
quality_df = pd.DataFrame(quality_metrics, index=method_names)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, metric in enumerate(['total_distance', 'smoothness', 'end_error']):
    axes[i].bar(method_names, quality_df[metric], alpha=0.7)
    axes[i].set_title(f'{metric.replace("_", " ").title()}')
    axes[i].set_ylabel('値')
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('\nサンプリング手法の品質比較:')
print(quality_df.round(4))

## 10. まとめと今後の改善点

### 実験結果のまとめ

このノートブックでは、個人最適化軌道生成のための拡散モデルの実装と可視化を行いました。

#### 主な成果：
1. **モデルアーキテクチャ**: UNetベースの拡散モデルを実装し、個人特性を条件として組み込みました
2. **ノイズスケジュール**: DDPMの線形βスケジュールを分析し、適切な信号対雑音比を確認しました
3. **訓練プロセス**: 短期デモ訓練で損失の収束を確認しました
4. **生成プロセス**: 逆拡散プロセスの段階的可視化により、ノイズから軌道への変換を観察しました
5. **個人特性の影響**: 異なる個人特性が生成軌道に与える影響を分析しました
6. **サンプリング比較**: DDPMとDDIMの比較により、効率と品質のトレードオフを評価しました

#### 今後の改善点：
1. **実データの使用**: 実際の軌道測定データでの訓練
2. **より長期の訓練**: 十分な収束までの訓練
3. **ハイパーパラメータ調整**: 学習率、ネットワーク構造、ノイズスケジュールの最適化
4. **評価メトリクス**: より詳細な軌道品質評価指標の導入
5. **条件付けの改善**: より効果的な個人特性の組み込み方法
6. **生成速度の向上**: より高速なサンプリング手法の探索

In [ ]:
# CLAUDE_ADDED
# 最終的な実験設定とパラメータの保存
experiment_summary = {
    'model_config': CONFIG,
    'total_parameters': total_params,
    'training_epochs': len(train_losses),
    'final_train_loss': train_losses[-1] if train_losses else None,
    'final_val_loss': val_losses[-1] if val_losses else None,
    'device': str(device),
    'sampling_methods_tested': method_names,
    'feature_correlations': condition_feature_corr.to_dict()
}

print('実験サマリー:')
for key, value in experiment_summary.items():
    if key != 'feature_correlations':  # 相関行列は長いので除外
        print(f'  {key}: {value}')

# 実験完了
print('\n🎉 可視化ノートブック完了!')
print('このノートブックを参考に、実際のデータでの訓練と更なる改善を行ってください。')